<div style="text-align: center;">
    <span style="font-size: 20px;"><b>Project 2</b></span>
</div>


<div style="text-align: right;">
    <span style="font-size: 20px;"><b>Date : 20/10/2023</b></span>
</div>


<span style="color: blue"><b>Rahulnath Mukkolla</b></span><br/>
<span style="color: blue"><b>11669483</b></span>

<span style="color: red"> Implement logistic regression using the algorithm provided in the book. Train your model with the training dataset once. The apply the model to the test data set. You will implement cross validation
later. Calculate F1 score and accuracy and submit your ipynb file.</span>

In [22]:
# Imported required libraries

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
import re
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression


In [23]:
# Reading the train, test, validation labels and text from the given github urls
# Reading the csv data using pndas library

given_testing_label =pd.read_csv('https://github.com/cardiffnlp/tweeteval/blob/main/datasets/sentiment/test_labels.txt?raw=true',header=None,names=['label'])
given_training_label =pd.read_csv('https://github.com/cardiffnlp/tweeteval/blob/main/datasets/sentiment/train_labels.txt?raw=true',header=None,names=['label'])
given_testing_text =pd.read_csv('https://github.com/cardiffnlp/tweeteval/blob/main/datasets/sentiment/test_text.txt?raw=true',sep='delimiter',header=None,names=['tweet'])
given_training_text =pd.read_csv('https://github.com/cardiffnlp/tweeteval/blob/main/datasets/sentiment/train_text.txt?raw=true',sep='delimiter',header=None,names=['tweet'])
val_label =pd.read_csv('https://github.com/cardiffnlp/tweeteval/blob/main/datasets/sentiment/val_labels.txt?raw=true',header=None,names=['label'])
val_text=pd.read_csv('https://github.com/cardiffnlp/tweeteval/blob/main/datasets/sentiment/val_text.txt?raw=true',sep='delimiter',header=None,names=['tweet'])

C:\Users\user\AppData\Local\Temp\ipykernel_4928\3218872862.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  given_testing_text =pd.read_csv('https://github.com/cardiffnlp/tweeteval/blob/main/datasets/sentiment/test_text.txt?raw=true',sep='delimiter',header=None,names=['tweet'])
C:\Users\user\AppData\Local\Temp\ipykernel_4928\3218872862.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  given_training_text =pd.read_csv('https://github.com/cardiffnlp/tweeteval/blob/main/datasets/sentiment/train_text.txt?raw=true',sep='delimiter',header=None,names=['tweet'])
C:\Users\user\AppData\Local\Temp\ipykernel_4

In [24]:
# List of suberddits 

subreddits = ['3DS.tsv', '4chan.tsv', '2007scape.tsv', 'ACTrade.tsv', 'amiugly.tsv',
                   'BabyBumps.tsv', 'baseball.tsv', 'canada.tsv', 'CasualConversation.tsv',
                   'DarknetMarkets.tsv', 'darksouls.tsv', 'elderscrollsonline.tsv', 'Eve.tsv',
                   'Fallout.tsv', 'fantasyfootball.tsv', 'GameDeals.tsv', 'gamegrumps.tsv',
                   'halo.tsv', 'Homebrewing.tsv', 'IAmA.tsv', 'india.tsv', 'jailbreak.tsv',
                   'Jokes.tsv', 'KerbalSpaceProgram.tsv', 'Keto.tsv', 'leagueoflegends.tsv',
                   'Libertarian.tsv', 'magicTCG.tsv', 'MakeupAddiction.tsv', 'Naruto.tsv',
                   'nba.tsv', 'oculus.tsv', 'OkCupid.tsv', 'Parenting.tsv', 'pathofexile.tsv',
                   'raisedbynarcissists.tsv', 'Random_Acts_Of_Amazon.tsv', 'science.tsv',
                   'Seattle.tsv', 'TalesFromRetail.tsv', 'talesfromtechsupport.tsv',
                   'ultrahardcore.tsv', 'videos.tsv', 'Warthunder.tsv', 'whowouldwin.tsv',
                   'xboxone.tsv', 'yugioh.tsv']

In [25]:
# Reading and loading the each suberddits, frequent words and adjectives files
# Separate the positive and nagative lexicons based on score and stored in seperate lists

positive=[]
negative=[]


files =['1850.tsv',
    '1860.tsv',
     '1870.tsv',
     '1880.tsv',
     '1890.tsv',
     '1900.tsv',
     '1910.tsv',
     '1920.tsv',
     '1930.tsv',
     '1940.tsv',
     '1950.tsv',
     '1960.tsv',
     '1970.tsv',
     '1980.tsv',
     '1990.tsv',
     '2000.tsv',
    ]

for sub in subreddits:
    subreddit = sub.split('.')[0]
    data_frame = pd.read_csv(f'subreddits/{sub}', sep='\t', header=None,names=['word','score1','score2'])
    positive.extend(data_frame.loc[data_frame["score1"] >data_frame["score2"], "word"].tolist())
    negative.extend(data_frame.loc[data_frame["score2"] >data_frame["score1"], "word"].tolist())


for sub in files:
    subreddit = sub.split('.')[0]
    data_frame = pd.read_csv(f'frequency/{sub}', sep='\t', header=None,names=['word','score1','score2'])
    positive.extend(data_frame.loc[data_frame["score1"] >data_frame["score2"], "word"].tolist())
    negative.extend(data_frame.loc[data_frame["score2"] >data_frame["score1"], "word"].tolist())
    data_frame = pd.read_csv(f'adjective/{sub}', sep='\t', header=None,names=['word','score1','score2'])
    positive.extend(data_frame.loc[data_frame["score1"] >data_frame["score2"], "word"].tolist())
    negative.extend(data_frame.loc[data_frame["score2"] >data_frame["score1"], "word"].tolist())


In [26]:
# 
import nltk
import re
from nltk.corpus import stopwords
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def feautures(tweet):
    tweetre=re.sub(r'[^\w]',' ',tweet)
    tokens=tweetre.split()
    ans=[]
    #urls
    ans.append(tweet.count('.com'))
    #hashtags
    ans.append(tweet.count('#'))
    ans.append(tweet.count('@'))
    ans.append(tweet.count('!'))
    cc=0
    for i in tokens:
        #print(i.lower())
        if i.lower() in positive:
            cc=cc+1
    #print(cc)
    if cc!=0:
        ans.append(np.log(cc))
    else:
        ans.append(0)
    cc=0
    for i in tokens:
        if i.lower() in negative:
            cc=cc+1
    #print(cc)
    if cc!=0:
        ans.append(np.log(cc))
    else:
        ans.append(0)
    emojis=[':)',':D',';)',':p','<3',':P']
    cc=0
    for i in emojis:
        if i in tweet:
            cc=cc+1
    ans.append(cc)
    ans.append(np.log(len(tokens)))
    maxi=0
    for i in tokens:
        maxi=max(maxi,len(i))
    ans.append(maxi)
    cc=0
    for i in tokens:
        if len(i)>5:
            cc=cc+1
    ans.append(cc)
    cc=0
    for i in tokens:
        if i.isupper():
            cc=cc+1
    ans.append(cc)
    cc=0
    #count stop words
    for i in tokens:
        if i in stop_words:
            cc=cc+1
    ans.append(cc)
    return ans

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
feautures(train_text.iloc[2]['tweet'])

[0,
 0,
 0,
 0,
 2.833213344056216,
 3.044522437723423,
 0,
 3.1780538303479458,
 9,
 5,
 1,
 9]

In [28]:
# Testing only 1000 records on each test
# Note: Taking large time to load the data so testing with 1000 records

given_training_text = given_training_text.head(1000)
given_testing_text = given_testing_text.head(1000)
given_training_label = given_training_label.head(1000)
given_testing_label = given_testing_label.head(1000)


In [29]:
train_data=[feautures(i) for i in given_training_text['tweet']]
test_data=[feautures(i) for i in given_testing_text['tweet']]
x_training_data=np.array(train_data)
x_testing_data=np.array(test_data)
y_training_data = np.array(given_training_label['label'])
y_testing_data = np.array(given_testing_label['label'])
model=LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [30]:
f1score =f1_score(y_testing_data, y_pred,average ='weighted')
accscore=accuracy_score(y_testing_data, y_pred)
print('F1 Score:', f1score*100)
print('Accuracy Score:', accscore*100)


F1 Score: 37.47636215917585
Accuracy Score: 45.300000000000004
